<a href="https://colab.research.google.com/github/kiddinn/timesketch/blob/colab_sauce/Stolen_Szechuan_Sauce_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Case of The Stolen Szechuan Sauce

This is a simple colab demonstrating one way of analyzing data from the Stolen Szechuan Sauce challenge (found [here](https://dfirmadness.com/the-stolen-szechuan-sauce/)).

This colab will not go into any of the data upload. It assumes that all data is already collected and uploaded to Timesketch. To see one way of uploading the data to Timesketch, use [this colab](https://colab.research.google.com/github/google/timesketch/blob/master/notebooks/Stolen_Szechuan_Sauce_Data_Upload.ipynb)

For a more generic instructions of Colab can be [found here](https://colab.research.google.com/github/google/timesketch/blob/master/notebooks/colab-timesketch-demo.ipynb)

If you are running this on a cloud runtime you'll need to install these dependencies:

In [ ]:
# @markdown Only execute if not already installed and running a cloud runtime
!pip install timesketch_api_client

In [ ]:
# @title Import libraries
# @markdown This cell will import all the libraries needed for the running of this colab.

import re

import pandas as pd

from timesketch_api_client import config

In [ ]:
# @markdown Get a copy of the Timesketch client object.
# @markdown Parameters to configure the client:
# @markdown + host_uri: https://demo.timesketch.org
# @markdown + username: demo
# @markdown + auth_mode: timesketch (username/password)
# @markdown + password: demo

ts_client = config.get_client()

Now that we've got a copy of the TS client we need to get to the sketch.

In [ ]:
for sketch in ts_client.list_sketches():
  if not sketch.name.startswith('Szechuan'):
    continue

  print('We found the sketch to use')
  print(f'[{sketch.id}] {sketch.name} - {sketch.description}')
  break

Now we can start answering the questions.

## Questions

### What’s the Operating System of the Server?

Let's start exploring this, OS information is stored in the registry. Let's query it

In [ ]:
cur_df = sketch.explore(
    'parser:"winreg/windows_version"',
    as_pandas=True,
    return_fields='datetime,key_path,data_type,message,timestamp_desc,parser,display_name,product_name,hostname,timestamp_desc'
)

In [ ]:
cur_df[['hostname', 'product_name']]

So we now have the all the data, we can read the data from the table or do one more filtering to get the answer:

In [ ]:
cur_df[cur_df.hostname == 'CITADEL-DC01'].product_name.value_counts()

### What’s the Operating System of the Desktop?

we can use the same data as we collected before:

In [ ]:
cur_df[cur_df.hostname == 'DESKTOP-SDN1RPT'].product_name.value_counts()

### What was the local time of the Server?

To answer that we need to get the current control set

In [ ]:
cur_df = sketch.explore(
    'HKEY_LOCAL_MACHINE*System*Select AND hostname:"CITADEL-DC01"',
    as_pandas=True,
    return_fields=(
        'datetime,key_path,data_type,message,timestamp_desc,parser,display_name,'
        'product_name,hostname,timestamp_desc,values')
)

Now let's look at what the value is set for the key.

In [ ]:
for key, value in cur_df[['key_path', 'values']].values:
  print(f'Key: {key}')
  print(f'Value: {value}')

We can parse this out a bit more if we want to, or just read from there that the current value is 1

In [ ]:
cur_df['current_value'] = cur_df['values'].str.extract(r'Current: \[[A-Z_]+\] (\d) ')

cur_df[['key_path', 'current_value']]

The current one is set 1

In [ ]:
cur_df = sketch.explore(
    'TimeZoneInformation AND hostname:"CITADEL-DC01"',
    as_pandas=True,
    return_fields='datetime,key_path,data_type,message,timestamp_desc,parser,display_name,product_name,hostname,timestamp_desc,configuration'
)
cur_df

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
cur_df[['configuration']]

So we need to extract what is in `TimeZoneKeyName`, we can do this differently. For now we can just read the configuration field, and then split it into a dict and then construct a new DataFrame with these fields, that is taking a line that is `key1: value1 key2: value2 ...` and creating a data frame with `key1, key2, ...` being the column names.

In [ ]:
lines = []
for value in cur_df['configuration'].values:
  items = value.split()
  line_dict = {}
  key = ''
  value_list = []
  for item in items:
    if item.endswith(':'):
      if key and value_list:
        line_dict[key] = ' '.join(value_list)
        value_list = []
      key = item[:-1]
      continue
    value_list.append(item)
  if key and value_list:
    line_dict[key] = ' '.join(value_list)
    
  lines.append(line_dict)
time_df = pd.DataFrame(lines)

Let's look at the newly constructed data frame

In [ ]:
time_df

Then we've got the time zone of the server, which is `Pacific Standard Time`

### Was there a breach?


**MISSING TEXT HERE**

### What was the initial entry vector (how did they get in)?

If we assume they got in from externally, doing some statistics on the network data might be useful. For that we need to do some aggregations.

First to understand what aggregations are available to use, and how to use them, let's use the `list_available_aggregators` which produces a data frame with the names of the aggregators and what parameters they need for configuration.


In [ ]:
sketch.list_available_aggregators()

Now that we know what aggregators are available, let's start with aggregating the field `Source`, and get the top 10.

For that we need to use the `field_bucket` aggregator, and configuring it using the parameters `field`, `limit` and `supported_charts`.

The charts that are available are:
 + barchart
 + hbarchart
 + table
 + circlechart
 + linechart

For this let's use a horizontal bar chart, `hbarchart`

In [ ]:
params = {
    'field': 'Source',
    'limit': 10,
    'supported_charts': 'hbarchart',
    'chart_title': 'Top 10 Source IP',
}

aggregation = sketch.run_aggregator(aggregator_name='field_bucket', aggregator_parameters=params)
aggregation.chart

If you prefer to get the data frame instead of the chart you can call `aggregation.table`

In [ ]:
aggregation.table

Now let's look at the `Destination` field, same as before:

In [ ]:
params = {
    'field': 'Destination',
    'limit': 10,
    'supported_charts': 'hbarchart',
    'chart_title': 'Top 10 Source IP',
}

aggregation = sketch.run_aggregator(aggregator_name='field_bucket', aggregator_parameters=params)
aggregation.chart

We can clearly see that the ```194.61.24.102``` sticks out, so lets try to understand what this IP did. Also note that it is not common that a system from the internet tries to connect to a intranet IP.

#### A Look at IP 194.61.24.102

In [ ]:
attacker_dst = sketch.explore(
    'Source:"194.61.24.102" AND data_type:"pcap:wireshark:entry"',
    as_pandas=True,
    return_fields='datetime,message,timestamp_desc,Destination,DST port,Source,Protocol,src port')
attacker_dst.head(10)

We got a fairly large table, let's look at the size:

In [ ]:
attacker_dst.shape

We will now need to do some aggregation on the data that we got, let's use pandas for that. For that there is a function called `groupby` where we can run aggregations.

We want to group based on `DST port` and `Destination`, so we only need those two columns + one more to store the count/sum.

In [ ]:
attacker_group = attacker_dst[['DST port','Destination', 'Protocol']].groupby(
    ['DST port','Destination'], as_index=False)

Now we got a group, and to get a count, we can use the `count()` function of the group.

In [ ]:
attacker_dst_mytable = attacker_group.count()
attacker_dst_mytable.rename(columns={'Protocol': 'Count'}, inplace=True)
attacker_dst_mytable.sort_values(by=['Count'], ascending=False)

So we can already point out that there is a lot of traffic from this ip to ```10.42.85.10``` on port ```3389```which is used for Remote Desktop Protocol (RDP)

Let's now look at the IP traffic as it was parsed by scapy

In [ ]:
attacker_dst = sketch.explore(
    '194.61.24.102 AND data_type:"scapy:pcap:entry"',
    as_pandas=True,
    return_fields='datetime,message,timestamp_desc,ip_flags,ip_dst,ip_src,payload,tcp_flags,tcp_seq,tcp_sport,tcp_dport,tcp_window')


Let's look at a few entries here:

In [ ]:
attacker_dst.head(10)

What we can see here is that quite a bit of the information is  in the message field that we need to decode.

We also see that the `evil` bit is set... we could query for that as well. Let's start there, to do an aggregation based on that.

In [ ]:
params = {
    'field': 'ip_src',
    'query_string': 'ip_flags:"evil"',
    'supported_charts': 'hbarchart',
    'chart_title': 'Source IPs with "evil" bit set',
}

aggregation = sketch.run_aggregator(aggregator_name='query_bucket', aggregator_parameters=params)
aggregation.table

We could even save this (if you have write access to the sketch, which the demo user does not have)

In [ ]:
name = 'Source IPs with "evil" bit set'
aggregation.name = name
aggregation.title = name
aggregation.save()

And now we could use this in a story for instance.

But let's move on and parse the message field:

First let's look at a single entry. To see how it is constructed:

In [ ]:
attacker_dst.iloc[0].message

Now that we know that, let's first remove the `<bound method...` in the beginning. Let's check to see if it's the same across the board:

In [ ]:
attacker_dst.message.str.slice(start=0, stop=30).unique()

OK, so it's the same, we can therefore just use the slice method to remove this part of the string. After that we can then split the string based on `|` which separates the protocols.

In [ ]:
attacker_packages = attacker_dst.message.str.slice(start=30).str.split('|', expand=True)

Let's explain what was done in the above syntax. First of all we used the [slice method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html) to cut the first 30 characters out of the messages field. What we are left with is the rest of the message string. We then use the [split method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html) to split the string, based on `|`, and adding the option of `expand=True`, which then expands the results into a separate dataframe (as an opposed to just a list).

Now let's look at how this looks like:

In [ ]:
attacker_packages.head(3)

We can see a lot of values there are marked as None.. and basically all the columns from 3 and up are not useful, so let's remove those. And then rename the remaining columns

In [ ]:
attacker_packages = attacker_packages[[0, 1, 2]]
attacker_packages.columns = ['ether', 'ip', 'transport']

And let's look at how this looks like now:

In [ ]:
attacker_packages.head(3)

Now let's look at what happened in the first few packages:

In [ ]:
attacker_packages[['transport']].head(10)

What we can see here is that there is first an ICMP (Ping) then two HTTP/HTTPS Requests , another ICMP and then the 3389 traffic begins.

We could obviously parse this even further if we want to.

In [ ]:
def parse_row(row):
  items = row.split()
  protocol = items[0][1:]
  line_dict = {
      'protocol': protocol
  }
  for item in items[1:]:
    key, _, value = item.partition('=')
    if key == 'options':
      # We don't want options nor anything after that.
      break
    line_dict[key] = value
  return line_dict

proto_df = pd.DataFrame(list(attacker_packages['transport'].apply(parse_row).values))

Let's look at it, but first let's add in the datetime, since these are the same records as we had in the original DF we can simply apply the datatime there.

In [ ]:
proto_df['datetime'] = attacker_dst['datetime']

In [ ]:
proto_df.head(3)

So now if we look at the first few actions made:

In [ ]:
proto_df[['datetime', 'protocol', 'type', 'dport']].head(10)

So you can see the first action here.

+ ICMP echo request
+ TCP HTTPS
+ TCP HTTP
+ ICMP timestamp request
+ ICMP echo reply
+ TCP Remote Desktop, 3389



Let's look at the pair of both IPs:

In [ ]:
attacker_dst = sketch.explore(
    '(194.61.24.102 AND 10.42.85.10) AND data_type:"scapy:pcap:entry"', 
    as_pandas=True,
    return_fields='datetime,message,timestamp_desc,ip_flags,ip_dst,ip_src,payload,tcp_flags,tcp_seq,tcp_sport,tcp_dport,tcp_window')
attacker_dst.head(10)

We can then do the same as we did before to break things down.

In [ ]:
attacker_packages = attacker_dst.message.str.slice(start=30).str.split('|', expand=True)
attacker_packages = attacker_packages[[0, 1, 2]]
attacker_packages.columns = ['ether', 'ip', 'transport']

proto_df = pd.DataFrame(list(attacker_packages['transport'].apply(parse_row).values))
proto_df['datetime'] = attacker_dst['datetime']

proto_df[['datetime', 'protocol', 'type', 'dport']].head(10)

So we know that this seems to be a RDP connection from the IP 194.61.24.102. Let's look at login events:

In [ ]:
evtx_df = sketch.explore(
    '194.61.24.102 AND data_type:"windows:evtx:record"',
    as_pandas=True,
    return_fields='datetime,message,timestamp_desc,event_identifier,hostname,strings,user_sid,username,source_name,computer_name'
)

In [ ]:
evtx_df.head(3)

Let's get a quick overview of the data:

In [ ]:
evtx_df.username.value_counts()

In [ ]:
evtx_df.event_identifier.value_counts()

In [ ]:
evtx_df.source_name.value_counts()

### Was malware used? If so what was it? If there was malware answer the following:
#### What process was malicious?


#### What IP Address is the malware calling to?
#### Where is this malware on disk?
#### When did it first appear?
#### Did someone move it?
#### What were the capabilities of this malware?
#### Is this malware easily obtained?
#### Was this malware installed with persistence on any machine?
##### When?
##### Where?


#### Identify the IP Address that delivered the payload.


In [ ]:
attacker_dst = sketch.explore(
    '(194.61.24.102 AND 10.42.85.10) AND data_type:"scapy:pcap:entry" AND *http* AND *GET*', 
    as_pandas=True,
    return_fields='datetime,message,timestamp_desc,ip_flags,ip_dst,ip_src,payload,tcp_flags,tcp_seq,tcp_sport,tcp_dport,tcp_window')
attacker_dst.head(10)

### What malicious IP Addresses were involved?
#### Were any IP Addresses from known adversary infrastructure?
#### Are these pieces of adversary infrastructure involved in other attacks around the time of the attack?


If we assume 10.42.0.0 is the internal network, lets see which connections are made from that internal network

In [ ]:
data = sketch.explore('ip_src:10.42.85* AND NOT ip_dst:10.42.85* AND data_type:"scapy:pcap:entry"', as_pandas=True,return_fields='datetime,ip_src,message,timestamp_desc,tcp_dport,ip_dst')



In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:

data[['datetime','ip_src', 'ip_dst', 'tcp_dport']]

mytable = data.groupby(['ip_src','ip_dst']).size().to_frame('count').reset_index()
mytable.sort_values(by=['count'], ascending=False)
mytable

194.61.24.102 also shows bad signals on Virustotal

### Did the attacker access any other systems?
#### How?
#### When?
#### Did the attacker steal or access any data?
##### When?


### What was the network layout of the victim network?

This is a question we might not be able to answer with Timesketch


### What architecture changes should be made immediately?


### Did the attacker steal the Szechuan sauce? If so, what time?


### Did the attacker steal or access any other sensitive files? If so, what times?


### Finally, when was the last known contact with the adversary?

## NOT DONE ANYTHING AFTER HERE... THER EARE ANSWERS HERE

In [ ]:
data = sketch.explore('secret', as_pandas=True)


In [ ]:
params = {
    'field': 'Source',
    'limit': 10,
    'supported_charts': 'hbarchart',
    'chart_title': 'Top 10 Source IP',
}

aggregation = sketch.run_aggregator(aggregator_name='field_bucket', aggregator_parameters=params)

In [ ]:
aggregation.chart

In [ ]:
params = {
    'field': 'Destination',
    'limit': 10,
    'supported_charts': 'hbarchart',
    'chart_title': 'Top 10 Source IP',
}

aggregation = sketch.run_aggregator(aggregator_name='field_bucket', aggregator_parameters=params)
aggregation.chart

In [ ]:
data

# Network statistics

In [ ]:
all_comms = sketch.explore('timestamp_desc:all_packets.csv', as_pandas=True,return_fields='Source,Destination')


In [ ]:
all_comms.info()

In [ ]:
#mytable = all_comms.groupby(['Source','Destination']).size()
mytable = all_comms.groupby(['Source','Destination']).size().to_frame('count').reset_index()
mytable.sort_values(by=['count'], ascending=False)

We can clearly see what are the common combinations, so we should have a look at `194.61.24.102` and further investigate



#194.61.24.102

Lets start with a simple explore for the ip but not network logs.

In [ ]:
attacker_ip_not_network = sketch.explore('194.61.24.102 AND NOT timestamp_desc:all_packets.csv', as_pandas=True,return_fields='datetime,message,timestamp_desc')


In [ ]:
attacker_ip_not_network = sketch.explore('194.61.24.102 AND NOT timestamp_desc:all_packets.csv', as_pandas=True,return_fields='datetime,message,timestamp_desc',max_entries=5)


In [ ]:
attacker_ip_not_network.info()

In [ ]:
attacker_ip_not_network

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
attacker_ip_not_network

In [ ]:
attacker_ip_not_network.style.set_properties(**{'text-align': 'left'})


In [ ]:
# What is the "attacker" accesing:

attacker_dst = sketch.explore('194.61.24.102 AND timestamp_desc:all_packets.csv', as_pandas=True,return_fields='datetime,message,timestamp_desc,Destination,DST port')
attacker_dst

In [ ]:
attacker_dst

In [ ]:
attacker_dst_mytable = attacker_dst.groupby(['DST port','Destination']).size().to_frame('count').reset_index()
attacker_dst_mytable.sort_values(by=['count'], ascending=False)

In [ ]:
sketch.list_available_aggregators()


# Ideas

We could add VT queries for the IPs etc to the colab